<a href="https://colab.research.google.com/github/sabyasm/ipython-notebooks/blob/master/sub_003_keras_base_learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.metrics import accuracy_score
import xgboost as xgb


# A host of Scikit-learn models
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.kernel_approximation import RBFSampler
from sklearn.pipeline import make_pipeline
from scipy.stats import randint

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models
from keras import layers
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score




Using TensorFlow backend.


# **Download Feature Data stored in pickle format**

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1KvpsAYprIzDISjY3pIltj8rTjhkBmq5C'})
download.GetContentFile('features.pkl')

#https://drive.google.com/open?id=1KvpsAYprIzDISjY3pIltj8rTjhkBmq5C

In [0]:
# Assuming pickle feats are already downloaded
import pickle
file = open('features.pkl', 'rb')
labels = [[],[]]
train_features = pickle.load(file)
test_features = pickle.load(file)
labels[0] = pickle.load(file)
labels[1] = pickle.load(file)
uid = pickle.load(file)
file.close()

In [0]:
xtrain, xtest, ytrain, ytest = train_features, test_features, np.array(labels[0]), np.array(labels[1])

In [7]:
xtrain.shape

(4640, 17415)

In [8]:
# Prediction score: 0.77606 ~ 76.something
##############################################
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(2017)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(2017)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(2017)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
#####################################################

def create_network():
    model = Sequential()
    model.add(Dense(1024, input_dim=17415, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid')) #sigmid .76

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, 
                              verbose=0, mode='auto')  
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, epochs=15,
          batch_size=512,validation_split=0.1 )  

kfold = StratifiedKFold(n_splits=5, shuffle=True)
                                 
# Evaluate neural network using three-fold cross-validation
results = cross_val_score(neural_network, xtrain, ytrain, cv=kfold,fit_params = {'callbacks': [earlystop]})        

neural_network.fit(xtrain,ytrain,verbose=0)
y_pred = neural_network.predict(xtest)

Train on 3339 samples, validate on 372 samples
Epoch 1/15
3339/3339 [==============================] - 2s 654us/step - loss: 0.6600 - acc: 0.6493 - val_loss: 0.5722 - val_acc: 0.7715
Epoch 2/15
3339/3339 [==============================] - 1s 224us/step - loss: 0.5473 - acc: 0.7281 - val_loss: 0.5052 - val_acc: 0.7500
Epoch 3/15
3339/3339 [==============================] - 1s 209us/step - loss: 0.5070 - acc: 0.7421 - val_loss: 0.4709 - val_acc: 0.7849
Epoch 4/15
3339/3339 [==============================] - 1s 207us/step - loss: 0.4761 - acc: 0.7670 - val_loss: 0.4640 - val_acc: 0.7661
Epoch 5/15
3339/3339 [==============================] - 1s 206us/step - loss: 0.4379 - acc: 0.7928 - val_loss: 0.4776 - val_acc: 0.7984
Epoch 6/15
3339/3339 [==============================] - 1s 206us/step - loss: 0.4303 - acc: 0.7972 - val_loss: 0.4823 - val_acc: 0.7608
Epoch 7/15
3339/3339 [==============================] - 1s 208us/step - loss: 0.3868 - acc: 0.8236 - val_loss: 0.4498 - val_acc: 0.7957
E

In [9]:
print(results)

[0.77610335 0.75       0.68857759 0.75000001 0.78640778]


In [10]:
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 75.02% (3.40%)


In [0]:
from google.colab import files
model_name="sub_002_keras-base-learner"

ts = pd.DataFrame(
{'Unique ID': uid,
 'label': y_pred.flatten()
})
ts.to_csv(model_name+".csv",index=False)
files.download(model_name+".csv") 

In [11]:
print("Prediction score: %.5f" % accuracy_score(y_pred, ytest))

Prediction score: 0.77606
